In [1]:
import pandas as pd
import bioframe
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from matplotlib.colors import LogNorm, Normalize
# https://stackoverflow.com/questions/48625475/python-shifted-logarithmic-colorbar-white-color-offset-to-center
class MidPointLogNorm(LogNorm):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        LogNorm.__init__(self, vmin=vmin, vmax=vmax, clip=clip)
        self.midpoint=midpoint
    def __call__(self, value, clip=None):
        result, is_scalar = self.process_value(value)
        x, y = [np.log(self.vmin), np.log(self.midpoint), np.log(self.vmax)], [0, 0.5, 1]
        return np.ma.array(np.interp(np.log(value), x, y), mask=result.mask, copy=False)

In [3]:
# import standard python libraries
import seaborn as sns
# import libraries for biological data analysis
from coolpuppy import coolpup as cp
import cooler
import bioframe
# only to get the "fall" colormap ...
import cooltools.lib.plotting

In [7]:
import scipy
from cooltools import expected_cis
from mpire import WorkerPool
from cooltools import insulation

# Load coolers to get insulations


In [5]:
# get our coolers ...?
clr_fnames = {
    "esc": "esc_microc.hg38.mapq_30.250.mcool",
    "hff": "hff_microc.hg38.mapq_30.250.mcool",
}

clrs = {}
binsize = 500

for k,f in clr_fnames.items():
    clrs[k] = cooler.Cooler(f"{f}::/resolutions/{binsize}")

In [9]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
# create a view with chromosome arms using chromosome sizes and definition of centromeres
hg38_arms_full = bioframe.make_chromarms(hg38_chromsizes,  hg38_cens)

# can do 1 chromosome (or arm) as well ..
included_arms = hg38_arms_full["name"].to_list()[:44] # all autosomal ones ...
hg38_arms = hg38_arms_full[hg38_arms_full["name"].isin(included_arms)].reset_index(drop=True)

In [10]:
windows = [5*binsize, 10*binsize, 50*binsize]

insulation_dict = {}
for k, clr in clrs.items():
    insulation_dict[k] = insulation(
        clr,
        windows,
        view_df=hg38_arms,
        verbose=True,
        nproc=8,
    )

INFO:root:Processing region chr5_p
INFO:root:Processing region chr2_p
INFO:root:Processing region chr6_p
INFO:root:Processing region chr3_p
INFO:root:Processing region chr4_p
INFO:root:Processing region chr8_p
INFO:root:Processing region chr1_p
INFO:root:Processing region chr7_p
INFO:root:Processing region chr5_q
INFO:root:Processing region chr8_q
INFO:root:Processing region chr4_q
INFO:root:Processing region chr7_q
INFO:root:Processing region chr6_q
INFO:root:Processing region chr2_q
INFO:root:Processing region chr3_q
INFO:root:Processing region chr9_p
INFO:root:Processing region chr10_p
INFO:root:Processing region chr11_p
INFO:root:Processing region chr1_q
INFO:root:Processing region chr12_p
INFO:root:Processing region chr13_p
INFO:root:Processing region chr13_q
INFO:root:Processing region chr9_q
INFO:root:Processing region chr10_q
INFO:root:Processing region chr12_q
INFO:root:Processing region chr14_p
INFO:root:Processing region chr14_q
INFO:root:Processing region chr11_q
INFO:root:

In [21]:
# Write the insulation track as a bigwig:
for k in clrs:
    # derive output name based on cooler's name
    kkk = clr_fnames[k]
    out_fname = ".".join( kkk.split(".")[:-1] )
    # apparently insulation sometimes reports the same bin
    df = insulation_dict[k].drop_duplicates(subset=["chrom","start","end"])
    for w in windows:
        # store in bigwig ...
        bioframe.to_bigwig(
            df,
            hg38_chromsizes,
            f"{out_fname}.insul.{w}.bw",
            value_field=f"log2_insulation_score_{w}",
        )